In [1]:
#default_exp lesson2

In [2]:
import git
import os

In [3]:
repo = git.Repo(search_parent_directories=True)
repo_root = repo.git.rev_parse("--show-toplevel")
os.chdir(repo_root)
os.getcwd()

'/Users/rforgione/development/fastai-part2'

In [4]:
#export
from solutions.lesson1 import *
from fastai.datasets import *
import torch
from torch import nn
import numpy as np
from functools import wraps

In [5]:
import pdb

In [6]:
??get_data

Signature: get_data(data_path)
Docstring: <no docstring>
Source:   
def get_data(data_path):
  with gzip.open(data_path, mode="rb") as d:
    ((X_train, y_train), (X_valid, y_valid), _) = pickle.load(d, encoding='latin-1')
    X_train, y_train, X_valid, y_valid = map(torch.tensor, (X_train, y_train, X_valid, y_valid))
    return (X_train, y_train, X_valid, y_valid)
File:      ~/development/fastai-part2/solutions/lesson1.py
Type:      function


In [7]:
#export
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'
path = download_data(MNIST_URL, ext=".gz")
X_train, y_train, X_valid, y_valid = get_data(path)

In [8]:
X_train.shape, X_valid.shape

(torch.Size([50000, 784]), torch.Size([10000, 784]))

In [9]:
y_train.shape, y_valid.shape

(torch.Size([50000]), torch.Size([10000]))

# Model Class

In [10]:
class Model(nn.Module):
    def __init__(self, layers):
        self._modules = {}
        for idx, layer in enumerate(layers):
            self._modules[f'layer{idx}'] = layer
        
    def __call__(self, x):
        for _, layer in self.named_children(): x = layer(x)
        return x

In [11]:
nh = 50
ni = X_train.shape[1]
c = int(y_train.max()) + 1

In [12]:
model = Model([
    nn.Linear(ni, nh),
    nn.ReLU(),
    nn.Linear(nh, c)
])

In [13]:
model

Model(
  (layer0): Linear(in_features=784, out_features=50, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=50, out_features=10, bias=True)
)

In [14]:
pred = model(X_train)

In [15]:
pred.shape, pred

(torch.Size([50000, 10]),
 tensor([[-0.0816,  0.0303, -0.1046,  ..., -0.1048, -0.0573,  0.0998],
         [ 0.0100, -0.0479, -0.0986,  ..., -0.1628, -0.0820,  0.1606],
         [-0.0561,  0.0262, -0.0808,  ..., -0.1641, -0.0790,  0.0569],
         ...,
         [-0.1743,  0.0557, -0.1266,  ..., -0.0885, -0.0888,  0.1244],
         [-0.1408, -0.0087, -0.1271,  ..., -0.0401, -0.1106,  0.1369],
         [-0.1068,  0.0717, -0.1291,  ..., -0.0958, -0.0599,  0.0835]],
        grad_fn=<AddmmBackward>))

# Log Softmax

First the naive way:

In [16]:
def log_softmax(vec):
    return (vec.exp() / vec.exp().sum(-1)).log()

In [17]:
assert log_softmax(torch.tensor([[1.,2.,3.]])).exp().sum() == 1

Then the numerically friendly way:

In [18]:
def log_softmax(vec):
    return vec - vec.exp().sum(-1).log()

In [19]:
assert log_softmax(torch.tensor([[1.,2.,3.]])).exp().sum() == 1

In [20]:
def nll(output, target):
    return -output[range(len(output)), target].mean() 

In [21]:
fake_output = torch.tensor([
    [0.5, 0.5],
    [0.3, 0.7]
])

fake_target = torch.tensor([
    0,
    1
])

In [22]:
log_first = np.log(0.5); log_first

-0.6931471805599453

In [23]:
log_second = np.log(0.7); log_second

-0.35667494393873245

In [24]:
correct_nll = -(log_first + log_second)/2

In [25]:
nll(torch.log(fake_output), fake_target)

tensor(0.5249)

In [26]:
assert torch.allclose(nll(fake_output.log(), fake_target), torch.tensor(correct_nll).float())

# LogSumExp

LogSumExp is a way of making the sum of exponents more numerically stable.

In [27]:
def logsumexp(vec):
    max_elem = vec.max()
    return max_elem + (vec - max_elem).exp().sum(-1).log()

In [28]:
assert torch.tensor([1,2,3]).float().exp().sum().log() == logsumexp(torch.tensor([1,2,3]).float())

This is a way of making the log of a sum of exponentials more numerically stable. You can take advantage of the fact that dividing two exponentials results in subtracting their exponents. You can factor out the largest exponential, which subtracts its exponent from each of the other exponents. Then you can take advantage of the log rules that state that log(ab) = log(a) + log(b), which allows you to add the highest exponential to the log of the sum of the others. 

In [29]:
assert logsumexp(torch.tensor([1,2,3]).float()) == torch.logsumexp(torch.tensor([1,2,3]).float(), dim=(0,))

## log_softmax in terms of LogSumExp

In [30]:
def log_softmax(vec):
    return vec - vec.logsumexp(dim=(0,), keepdim=True)

In [31]:
log_softmax(torch.tensor([1,2,3]).float()) == torch.log_softmax(torch.tensor([1,2,3]).float(), dim=0)

tensor([ True,  True, False])

In [32]:
np.array([[1,2,3], [3,4,5], [5,6,7]])[:, [0,1,2]]

array([[1, 2, 3],
       [3, 4, 5],
       [5, 6, 7]])

In [33]:
grad_example = torch.tensor([
    [0,0,1],
    [1,0,0],
    [0,1,0],
])

In [34]:
grad_example.argmax(dim=1)

tensor([2, 0, 1])

In [35]:
grad_example[range(len(grad_example)), grad_example.argmax(dim=1)] = 2

In [36]:
grad_example

tensor([[0, 0, 2],
        [2, 0, 0],
        [0, 2, 0]])

In [37]:
grad_example.nonzero()

tensor([[0, 2],
        [1, 0],
        [2, 1]])

In [38]:
grad_example[grad_example.nonzero()]

tensor([[[0, 0, 2],
         [0, 2, 0]],

        [[2, 0, 0],
         [0, 0, 2]],

        [[0, 2, 0],
         [2, 0, 0]]])

In [39]:
t = torch.tensor([
    [0,0,0,1,2],
    [3,5,3,0,0]
])

In [40]:
nonzero = t.nonzero()
xs = nonzero[:,0]
ys = nonzero[:,1]
t[xs, ys]

tensor([1, 2, 3, 5, 3])

In [41]:
tz = torch.zeros_like(t)
tz[xs, ys] = 1

In [42]:
tz

tensor([[0, 0, 0, 1, 1],
        [1, 1, 1, 0, 0]])

In [43]:
class LogSoftmax(nn.Module):
    def forward(self, x):
        self.input = x
        self.output = torch.log_softmax(self.input, dim=-1)
        return self.output
    
    def backward(self): 
        correct_labels = (self.output.grad > 0).float()
        msg = "LogSoftmax must be used with CE loss, found more than one output dim with nonzero gradients for the same example"
        assert correct_labels.sum(axis=0).max() == 1, msg
        self.input.grad = (correct_labels - self.output) * self.output.grad

In [44]:
mdl = Model([
    nn.Linear(784,100),
    nn.ReLU(),
    nn.Linear(100, c),
    LogSoftmax()
])
X_train.shape, mdl(X_train).shape

(torch.Size([50000, 784]), torch.Size([50000, 10]))

In [45]:
mdl2 = Model([
    nn.Linear(784,100),
    nn.ReLU(),
    nn.Linear(100, c),
    nn.LogSoftmax()
])

In [46]:
torch.exp(mdl(X_train))

tensor([[0.1018, 0.1082, 0.1049,  ..., 0.1113, 0.0977, 0.0842],
        [0.0997, 0.1145, 0.0936,  ..., 0.1044, 0.0988, 0.0877],
        [0.1007, 0.0976, 0.0882,  ..., 0.0924, 0.1099, 0.1043],
        ...,
        [0.1068, 0.1060, 0.0790,  ..., 0.1042, 0.1028, 0.0883],
        [0.1077, 0.0857, 0.0884,  ..., 0.1022, 0.1068, 0.1010],
        [0.1066, 0.0919, 0.0892,  ..., 0.1022, 0.1001, 0.0900]],
       grad_fn=<ExpBackward>)

In [47]:
loss = nn.NLLLoss()

In [48]:
l = loss(mdl(X_train), y_train)

In [49]:
l.backward()

In [50]:
a = torch.tensor([[1,2,3]]).float()
l = LogSoftmax()
l.forward(a)
l.output.grad = torch.tensor([[0,0,10]]).float()
l.backward()

In [51]:
torch.log_softmax(a, 1)

tensor([[-2.4076, -1.4076, -0.4076]])

In [52]:
print(a.grad)

tensor([[ 0.0000,  0.0000, 14.0761]])


In [53]:
assert torch.allclose(log_softmax(torch.tensor([1,2,3]).float()), torch.log_softmax(torch.tensor([1,2,3]).float(), dim=0))

In [54]:
print(list(mdl2.named_children()))

[('layer0', Linear(in_features=784, out_features=100, bias=True)), ('layer1', ReLU()), ('layer2', Linear(in_features=100, out_features=10, bias=True)), ('layer3', LogSoftmax())]


In [55]:
def fit(model, bs=64, lr=1e-3, epochs=1):
    n_batches = X_train.shape[0] // bs + 1
    losses = []
    for epoch in range(epochs):
        for i in range(n_batches):
            start_idx = i * bs
            end_idx = (i + 1) * bs
            X_b, y_b = X_train[start_idx:end_idx], y_train[start_idx:end_idx] 
            pred = model(X_b)
            loss = nll(pred, y_b)
            losses.append(loss.item())

            loss.backward()

            with torch.no_grad():
                for _, layer in model.named_children():
                    if hasattr(layer, 'weight'):
                        layer.weight.sub_(layer.weight.grad * lr)
                        layer.bias.sub_(layer.bias.grad * lr)
                        layer.zero_grad()
    print(losses[-1])

In [56]:
mdl

Model(
  (layer0): Linear(in_features=784, out_features=100, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=100, out_features=10, bias=True)
  (layer3): LogSoftmax()
)

In [57]:
for i,l in mdl.named_children(): 
    if hasattr(l, 'weight'): 
        print('has it')

has it
has it


In [58]:
fit(mdl2, 64, epochs=10)

/Users/rforgione/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


1.1718132495880127


In [59]:
fit(mdl, 64, epochs=100)

0.41941094398498535


In [60]:
def nograd(f):
    @wraps(f)
    def inner(*args, **kwargs):
        with torch.no_grad():
            return f(*args, **kwargs)
    return inner

In [61]:
class Optimizer:
    def __init__(self, model, lr):
        self.lr = lr
        params = []
        for _, layer in model.named_children():
            if hasattr(layer, 'weight'):
                params.append(layer)
        self.params = params
    
    @nograd
    def step(self):
        for param in self.params:
            param.weight.sub_(param.weight.grad * self.lr)
            param.bias.sub_(param.bias.grad * self.lr)
                
    @nograd
    def zero_grad(self):
        for param in self.params:
            param.zero_grad() 

In [63]:
def fit(model, optimizer, bs=64, epochs=1):
    n_batches = X_train.shape[0] // bs
    losses = []
    for epoch in range(epochs):
        for i in range(n_batches):
            start_idx = i * bs
            end_idx = (i + 1) * bs
            X_b, y_b = X_train[start_idx:end_idx], y_train[start_idx:end_idx] 
            pred = model(X_b)
            loss = nll(pred, y_b)
            losses.append(loss.item())

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    print(losses[-1])

In [74]:
def basic_model(nh, torch_softmax=False):
    return nn.Sequential(
        nn.Linear(784, nh),
        nn.ReLU(),
        nn.Linear(nh, 10),
        nn.LogSoftmax() if torch_softmax else LogSoftmax()
    )

In [73]:
mdl = basic_model(100)
opt = Optimizer(mdl, 1e-3)
fit(mdl, opt, epochs=10)

0.7323116660118103


In [75]:
mdl2 = basic_model(100, True)
opt2 = Optimizer(mdl2, 1e-3)
fit(mdl2, opt2, epochs=10)

/Users/rforgione/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.7221540212631226


In [76]:
class Dataset:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.x.shape[0]

In [77]:
d = Dataset(X_train, y_train)

In [78]:
d[2:5]

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([4, 1, 9]))

In [79]:
g = [(1,2), (3,4), (5,6)]

In [80]:
list(zip(*g))

[(1, 3, 5), (2, 4, 6)]

In [81]:
class SequentialSampler:
    def __init__(self, ds):
        self.ds = ds
        
    def __iter__(self):
        for i in range(len(self.ds)):
            yield(i)
            
    def __len__(self):
        return len(self.ds)

In [82]:
class DataLoader:
    def __init__(self, ds, bs, sampler=None):
        self.ds = ds
        self.bs = bs
        
        if callable(sampler):
            self.sampler = sampler(ds)
        else:
            self.sampler = sampler or SequentialSampler(ds)
    
    def __iter__(self):
        it = iter(self.sampler)
        nbatches = len(self.sampler)//self.bs + 1
        for _ in range(nbatches):
            try:
                idxs = []
                for _ in range(self.bs):
                    idxs.append(next(it))
                yield self.ds[idxs]
            except StopIteration:
                yield self.ds[idxs] # flush the final partial batch


In [83]:
dl = DataLoader(d, 64)

In [84]:
next(iter(dl))[0].shape

torch.Size([64, 784])

In [85]:
min([len(yb) for xb,yb in dl]), max([len(yb) for xb,yb in dl])

(16, 64)

In [86]:
def fit(model, optimizer, dl, epochs=1):
    losses = []
    for _ in range(epochs):
        for X_b, y_b in dl:
            pred = model(X_b)
            loss = nll(pred, y_b)
            losses.append(loss.item())

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    print(losses[-1])

In [87]:
mod = basic_model(100)

In [88]:
opt = Optimizer(mod, 1e-3)

In [89]:
fit(mod, opt, dl, 100)

0.42209887504577637


# Callbacks

In [95]:
class Callback:
    _order = 0
    
    def on_train_start(self):
        pass
        
    def on_epoch_start(self): 
        pass
    
    def on_batch_start(self): 
        pass
    
    def on_forward_start(self): 
        pass
    
    def on_forward_end(self): 
        pass
    
    def on_backward_start(self): 
        pass
    
    def on_backward_end(self): 
        pass
        
    def on_batch_end(self):
        pass
        
    def on_epoch_end(self):
        pass
    
    def on_train_end(self):
        pass

In [101]:
class Learn:
    def __init__(self, model, optimizer, data, loss):
        self.model = model
        self.optimizer = optimizer
        self.data = data
        self.loss = loss

In [96]:
# class CallbackHandler:
#     def __init__(self, learn, callbacks, callback_fns):
#         self.callbacks = [cb(learn) for cb in callback_fns] + callbacks 
        
#     def on_train_start(self):
#         for callback in self.callbacks: callback.on_train_start()
        
#     def on_epoch_start(self):
#         for callback in self.callbacks: callback.on_epoch_start()
        
#     def on_batch_start(self): 
#         for callback in self.callbacks: callback.on_batch_start()
            
#     def on_forward_start(self): 
#         for callback in self.callbacks: callback.on_forward_start()
            
#     def on_forward_end(self): 
#         for callback in self.callbacks: callback.on_forward_end()
            
#     def on_backward_start(self): 
#         for callback in self.callbacks: callback.on_backward_start()
            
#     def on_backward_end(self): 
#         for callback in self.callbacks: callback.on_backward_end()
        
#     def on_batch_end(self):
#         for callback in self.callbacks: callback.on_batch_end()
        
#     def on_epoch_end(self):
#         for callback in self.callbacks: callback.on_epoch_end()
        
#     def on_train_end(self):
#         for callback in self.callbacks: callback.on_train_end()

In [100]:
# class CallbackHandler(Callback):
#     def __init__(self, learn, callbacks, callback_fns):
#         self.callbacks = [cb(learn) for cb in callback_fns] + callbacks 
        
#     def __getattr__(self, attr):
#         if attr.startswith('on_'):
#             for callback in sorted(self.callbacks, key=lambda x: x._order): 
#                 f = getattr(callback, attr) 
#                 if f and f(): return False # false means don't stop

In [102]:
class ModelMode:
    TRAIN = 0
    VALID = 1
    TEST = 2

In [104]:
class DataBunch:
    def __init__(self, train_dl, valid_dl, test_dl=None, c=None):
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.test_dl = test_dl
        self.c = c

In [105]:
class Runner(object): 
    def __init__(self, learn, cbs=None, cb_funcs=None):
        self.learn = learn
        cb_funcs = cb_funcs or []
        cbs = cbs or []
        self.cbs = cb + [cb_func(learn) for cb_func in cb_funcs]
        
    def train(self):
        self.mode = ModelMode.TRAIN
        
    def valid(self):
        self.mode = ModelMode.VALID
        
    @property
    def mode(self):
        return self.mode 
        
    def one_batch(self, xb, yb):
        self.xb, self.yb = xb, yb
        if self('on_batch_start'): return
        pred = self.learn.model(xb)
        self.loss = self.learn.loss(pred, yb)
        
        if self.mode() == ModelMode.TRAIN:
            if not self('on_step_start'): optimizer.step()
            optimizer.zero_grad()
            self('on_step_end') 
        
    def all_batches(self):
        if self('on_epoch_start'): return
        self.epochs += 1
        dl = self.learn.data.train_dl if self.mode == ModelMode.TRAIN else self.learn.data.valid_dl
        for xb, yb in dl:
            self.xb, self.yb = xb, yb
            if not self('on_batch_start'): self.one_batch(xb, yb)
            self('on_batch_end')
        self('on_epoch_end') 
        
    def fit(learn, cb_handler=None, epochs=1):
        self('on_train_start')
        self.epochs = 0
        for _ in range(epochs):
            self.train()     
            self.all_batches()
            
            self.valid()
            self.all_batches()
        self('on_train_end')
        
    def __call__(self, name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, name, None)
            if f and f(): return True
        return False 